# CNN Practice 

### Google Cloud GPU use for computation
Environment: Container Image (Dockerfile) on Google Cloud

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# Good way for books and papers on Deep Learning
#https://arxiv.org/abs/2009.05673

In [2]:
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import glob
import cv2 as cv
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *

In [3]:
DATA_URL = "C:/Users/user1/Desktop/Image Processing/CNN Practice/seg_train/seg_train"
DATA_URL_test = "C:/Users/user1/Desktop/Image Processing/CNN Practice/seg_train/seg_train"

In [ ]:
# This directory was used for my docker container
#DATA_URL = "/tf/CNN Practice/seg_train/seg_train"
#DATA_URL_test = "/tf/CNN Practice/seg_test/seg_test"

In [4]:
# Image Shape
s = 150

In [5]:
# Makes the code labels 
code = {"buildings": 0, "forest": 1,"glacier":2,"mountain":3,"sea":4,"street":5}

In [ ]:
# What this program does is that it gets the folders from the url and then gets the image files using glob glob
# Then in the collection of files I pick a single file then read it
# Then I resize it ito shape (image, Width, Height,channel) ~ No need to specify channel
# Then add the image array as a list in X_train to create small lists in lits which is a 2D array 
# Then add the code labels to y_train

In [6]:
# Loads the Image data from directory and resizes it to collective list of Training data with lables
X_train = []
y_train = []

for folder in os.listdir(DATA_URL):
    files = glob.glob(str(DATA_URL+"/"+folder+"/*.jpg"))
    #print(f"Training data of {folder}:",len(files))
    for file in files:
        image = cv.imread(file)
        image_array = cv.resize(image,(s,s))
        X_train.append(list(image_array))
        y_train.append(code[folder])


In [ ]:
print(len(X_train))

In [ ]:
# Loads the Image data from directory and resizes it to collective list of Test data with labels
X_test = []
y_test = []

for folder in os.listdir(DATA_URL_test):
    files_1 = glob.glob(str(DATA_URL_test+"/"+folder+"/*.jpg"))
    #print(f"Training data of {folder}:",len(files))
    for file_1 in files_1:
        im = cv.imread(file_1)
        image_array_1 = cv.resize(im,(s,s))
        X_test.append(list(image_array_1))
        y_test.append(code[folder])


In [ ]:
# Gets index and value by using enumerate 

In [ ]:
def getcode(n):
    for x,y in code.items():
        if n == y:
            return x

In [ ]:
# Visualises the images with the corresponding code from training data
plt.figure(figsize=(20,20))
for n, i in enumerate(list(np.random.randint(0,len(X_train),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_train[i])    
    plt.axis('off')
    plt.title(getcode(y_train[i]))

In [ ]:
# Visuslises the images with the corresponding code from test data
plt.figure(figsize=(20,20))
for m, j in enumerate(list(np.random.randint(0,len(X_test),36))):
    plt.subplot(6,6,m+1)
    plt.imshow(X_test[j])
    plt.axis('off')
    plt.title(getcode(y_test[j]))

In [ ]:
# Sets them to numpy arrays
X_train = np.array(X_train,dtype=np.float32)
X_test = np.array(X_test,dtype=np.float32)
y_train = np.array(y_train,dtype=np.float32)
y_test = np.array(y_test,dtype=np.float32)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)
print(X_test.shape)
print(f'Bytes stored:{sys.getsizeof(X_train)} and 3.78GB')

X_train = X_train / 255
X_test = X_test / 255

In [ ]:
# Data (150,150,3)
# Padding adds fake white pixels on edge to make sure that it evenly analyzes each pixel
# Strides 
#Kernels is the Shape in which it is reading (3,3)
# Mostly deal with the square of numbers like 9,16,25


In [ ]:
# Creates Model with CNN model with Sequential API 
model = keras.Sequential()
model.add(Conv2D(512,kernel_size=(3,3),activation='relu',input_shape=(s,s,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(6,activation='softmax'))
# May want to add more layers to reduce image to smallest output

In [ ]:
# Compiles the model
model.compile(optimizer ='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
X_train.dtype

In [ ]:
history = model.fit(X_train,y_train,epochs=10,batch_size=50,validation_split = 0.2,verbose=1)

In [ ]:
#OOM stands for "out of memory". Your GPU is running out of memory, so it can't allocate memory for this tensor. There are a few things you can do:

#1. Decrease the number of filters in your Dense, Conv2D layers
#2. Use a smaller batch_size (or increase steps_per_epoch and validation_steps)
#3. Use grayscale images (you can use tf.image.rgb_to_grayscale)
#4.Reduce the number of layers
#5.Use MaxPooling2D layers after convolutional layers
#6.Reduce the size of your images (you can use tf.image.resize for that)
# 7. Use smaller float precision for your input, namely np.float32
#8.If you're using a pre-trained model, freeze the first layers (like this)